# Questo notebook importa le raccomandazioni degli analisti da marketscreener, di default il link è di intesa san paolo ma può essere cambiato come riportato sotto

In [207]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/INVEST-IO


In [208]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import numpy as np

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (109 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [209]:
import importlib
from textblob import TextBlob
# re = python module for regular expression
import re
from datetime import datetime, timedelta
from google.colab import files

# # # # TWITTER STREAM LISTENER - Analyzer # # # #
# we can use TextBlob for tweet sentiment analysis and determine the polarity of tweet, from -3 to 3;
# -3 = negative, 0 = neutral, 3 = positive

 
class DescAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    # claen_tweet using the re module to clean tweets text 
    def clean_tweet(self, desc_text):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", desc_text).split())
    def analyze_sentiment(self, desc_text):
        analysis = TextBlob(self.clean_tweet(desc_text))
        
        if analysis.sentiment.polarity > 0:
            return 3
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -3

In [210]:
def get_tds():
  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  #loop sulle raccomandazioni dalla pagina principale
  for i in wd.find_elements_by_css_selector('table#ALNI4.tabBody >* tr'):
    date = i.find_elements_by_css_selector('.pleft5.pright5.large30')[0].text
    reccom = i.find_elements_by_css_selector('.newsColCT.ptop3')[0].text
    link = i.find_elements_by_css_selector('.newsColCT.ptop3 > a')[0].get_attribute('href')
    output.append({'date': date, 'reccom' : reccom , 'link' : link})

In [211]:
def click_on_button(x):
  element = wd.find_element_by_css_selector(x)
  wd.execute_script("arguments[0].click();", element)

In [212]:
#MAIN
url = 'https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news-broker-research/' #< sostituisci qui il link
wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

output = []
next_button = 1

while next_button > 0:
  get_tds()
  try:
    button_container = 'a.nPageEndTab'
    wd.find_element_by_css_selector(button_container)
    click_on_button(button_container)
  except:
    next_button = 0

In [213]:
full_output = []
import time
def loop_single_reccomm(k):
  url = k['link']
  date = k['date']
  title = k['reccom']
  wd.get(url)
  time.sleep(abs(np.random.normal(2,1)))
  time_published = wd.find_element_by_css_selector('meta[itemprop="datePublished"]').get_attribute('content')
  desc = wd.find_element_by_id('grantexto').text
  full_output.append({'date_short' : date, 'title' : title, 'url': url, 'date_long' : time_published, 'desc' : desc})
  return full_output

for i in tqdm(output):
  loop_single_reccomm(i)
print (full_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



[{'date_short': '09/16', 'title': 'APPLE INC : Bernstein sticks Neutral', 'url': 'https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/APPLE-INC-Bernstein-sticks-Neutral-31308050/', 'date_long': '2020-09-16T13:38:03-04:00', 'desc': 'Bernstein analyst Toni Sacconaghi maintains his Neutral opinion on the stock. The target price is unchanged at USD 100.'}, {'date_short': '09/16', 'title': 'APPLE INC : Barclays reaffirms its Neutral rating', 'url': 'https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/APPLE-INC-Barclays-reaffirms-its-Neutral-rating-31305489/', 'date_long': '2020-09-16T08:13:04-04:00', 'desc': 'In a research note, Barclays analyst Tim Long has maintained his recommendation on the stock with a Neutral rating. The target price continues to be set at USD 100.'}, {'date_short': '09/16', 'title': 'APPLE INC : Credit Suisse reiterates its Neutral rating', 'url': 'https://www.marketscreener.com/quote/stock/APPLE-INC-4849/news/APPLE-INC-Credit-Suisse-reiterates

In [214]:
from pandas import DataFrame
dataset = DataFrame (full_output)
from tqdm import tqdm # This is an awesome package for tracking for loops
import pandas as pd

In [218]:
#add the new column 'sentiment' by passing the class TwitterAnalyzer
desc_analyzer = DescAnalyzer()

#sentiment analysis

polarity = lambda x: TextBlob(x).sentiment.polarity
subjectivity = lambda x: TextBlob(x).sentiment.subjectivity

dataset['polarity'] = dataset['desc'].astype(str).apply(polarity)
dataset['subjectivity'] = dataset['desc'].astype(str).apply(subjectivity)
dataset['sentiment'] = np.array([desc_analyzer.analyze_sentiment(text_desc) for text_desc in dataset['desc']])

dataset.head()

dataset.to_csv('sr_market_screener_reccom' + '_AAPL_YTD' + '.csv', index=False)
files.download('sr_market_screener_reccom' + '_AAPL_YTD' + '.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>